# Preparación

In [1]:
pip install --upgrade scikit-learn

In [2]:
pip install scikit-tda

     |████████████████████████████████| 72 kB 441 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 47 kB 4.5 MB/s 
     |████████████████████████████████| 125 kB 11.0 MB/s 
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18119 sha256=d5d72df0b46ad1e07e0eaa48b04786649515111b449c835b7de2f00af569a628
  Stored in directory: /root/.cache/pip/wheels/d2/9f/a8/67f1b86e47cd17338d3d07939f4660378e65b758c4594f96e3
  Created wheel for ripser: filename=ripser-0.6.1-cp37-cp37m-linux_x86_64.whl size=455259 sha256=d5f71fafdd0bc27f1b40dae15325468c009618bbbf5cfa8b4393e7d2c5f7fcba
  Stored in directory: /root/.cache/pip/wheels/6b/e3/c8/209a9df47cc9af8c6d2dbee4db558dc129494d3f4fbd670285
Successfully built hopcroftkarp ripser


In [3]:
pip install umap-learn

     |████████████████████████████████| 86 kB 3.0 MB/s 
     |████████████████████████████████| 1.1 MB 55.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82709 sha256=490094c3c8354395c823863521f6852c3321d962291618a0ad4bba35ea4a4e3b
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.5-py3-none-any.whl size=52603 sha256=9c007b86f29c50854cbc98505667fbf8c7a56068d21e99c0e511521a72596b2d
  Stored in directory: /root/.cache/pip/wheels/af/e9/33/04db1436df0757c42fda8ea6796d7a8586e23c85fac355f476
Successfully built umap-learn pynndescent


In [4]:
pip install gudhi

     |████████████████████████████████| 28.2 MB 23 kB/s 


In [5]:
import time
import warnings
from ripser import ripser
import numpy as np
import matplotlib.pyplot as plt
from persim import plot_diagrams
import persim
import glob
from numpy import asarray
from PIL import Image
import os

from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
import random
from sklearn.datasets import load_breast_cancer

from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
import warnings

import umap

import pandas as pd

warnings.filterwarnings('ignore')

In [6]:
!wget https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/EacFPSkXU9NMiMaWO_a9bfoB1f4h_3ibFIUvmF_8f7tNXA?download=1 -O ionosphere.csv

--2021-11-16 16:34:42--  https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/EacFPSkXU9NMiMaWO_a9bfoB1f4h_3ibFIUvmF_8f7tNXA?download=1
Resolving unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)... 40.108.195.27
Connecting to unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)|40.108.195.27|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/adines_unirioja_es/Documents/ionosphere.csv [following]
--2021-11-16 16:34:43--  https://unirioja-my.sharepoint.com/personal/adines_unirioja_es/Documents/ionosphere.csv
Reusing existing connection to unirioja-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 76467 (75K) [application/octet-stream]
Saving to: ‘ionosphere.csv’

ionosphere.csv      100%[===================>]  74.67K   402KB/s    in 0.2s    

2021-11-16 16:34:43 (402 KB/s) - ‘ionosphere.csv’ saved [76467/76467]



In [7]:
def load_prima():
  df=pd.read_csv("ionosphere.csv", sep=',',header=None)
  df1 = df.iloc[:,:-1]
  df2=df.iloc[:,-1]
  return df1.values,df2.values

In [8]:
def preparar_dataset(data,target,n_labeled=25):
  unos=np.where(target==1)[0]
  ceros=np.where(target==0)[0]

  datos_uno=np.array(data)[unos]
  datos_cero=np.array(data)[ceros]

  unos_valid=int((len(list(unos))-n_labeled)*0.2)
  ceros_valid=int((len(list(ceros))-n_labeled)*0.2)

  random.seed(15)
  indices_unos=random.sample(list(unos),n_labeled+unos_valid)

  random.seed(10)
  indices_ceros=random.sample(list(ceros),n_labeled+ceros_valid)


  puntos_unos=np.array(data)[indices_unos[0:n_labeled]]
  puntos_ceros=np.array(data)[indices_ceros[0:n_labeled]]

  puntos_unos_valid=np.array(data)[indices_unos[n_labeled:]]
  puntos_ceros_valid=np.array(data)[indices_ceros[n_labeled:]]

  unlabeled_unos=list(set(unos)-set(indices_unos))
  unlabeled_ceros=list(set(ceros)-set(indices_ceros))
  unlabeled=unlabeled_unos+unlabeled_ceros
  X_unlabeled_unos=np.array(data)[unlabeled_unos]
  X_unlabeled_ceros=np.array(data)[unlabeled_ceros]

  return (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)

In [9]:
def analizar_punto(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.bottleneck(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.bottleneck(dgms_unos_mod, dgms_unos, matching=False)
  clase=-1

  if distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [10]:
def analizar_punto_th(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0.8,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.bottleneck(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.bottleneck(dgms_unos_mod, dgms_unos, matching=False)

  distancia=distance_bottleneck_cero+distance_bottleneck_uno

  if max(distance_bottleneck_cero/distancia,distance_bottleneck_uno/distancia)<th:
    clase=-1
  elif distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [11]:
def analizar_punto_Wasserstein(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.wasserstein(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.wasserstein(dgms_unos_mod, dgms_unos, matching=False)
  clase=-1

  if distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [12]:
def analizar_punto_Wasserstein_th(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0.8,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.wasserstein(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.wasserstein(dgms_unos_mod, dgms_unos, matching=False)

  distancia=distance_bottleneck_cero+distance_bottleneck_uno

  if max(distance_bottleneck_cero/distancia,distance_bottleneck_uno/distancia)<th:
    clase=-1
  elif distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [13]:
def analizar_puntos(data,target,func,th=0,reduccion=False,dim=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target)
  embedding=data
  if reduccion==True:
    embedding=umap.UMAP(random_state=75).fit_transform(data)
  (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target)


  bien=0
  mal=0
  dudoso=0

  puntos_ceros_final=puntos_ceros.tolist()
  puntos_unos_final=puntos_unos.tolist()
  puntos_dudosos_final=[]

  dgms_ceros = ripser(puntos_ceros_umap)['dgms'][dim]
  dgms_unos = ripser(puntos_unos_umap)['dgms'][dim]
  for i,punto in enumerate(X_unlabeled_unos_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,dgms_ceros,dgms_unos,th,dim)
    if clase==1:
      bien=bien+1
      puntos_unos_final.append(X_unlabeled_unos[i])
    elif clase==0:
      mal=mal+1
      puntos_ceros_final.append(X_unlabeled_unos[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_unos[i])


  
  for i,punto in enumerate(X_unlabeled_ceros_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,dgms_ceros,dgms_unos,th,dim)
    if clase==1:
      mal=mal+1
      puntos_unos_final.append(X_unlabeled_ceros[i])
    elif clase==0:
      bien=bien+1
      puntos_ceros_final.append(X_unlabeled_ceros[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_ceros[i])
  
  comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

  return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final))

In [14]:
def resumen(bien,mal,dudosos):
  print('-----------------RESUMEN------------------')
  print('Bien anotados '+str(bien))
  print('Mal anotados '+str(mal))
  print('Sin anotar '+str(dudosos))
  print('')
  if bien+mal!=0:
    print('Porcentaje correcto '+str(bien/(bien+mal)))
  else:
    print('Porcentaje correcto 0')
  print('Porcentaje anotado '+str((bien+mal)/(bien+mal+dudosos)))

In [15]:
import gudhi as gd  

def obtener_radio(puntos):
  skeleton = gd.RipsComplex(points = puntos, max_edge_length = 90000000000)
  simplex_tree = skeleton.create_simplex_tree()
  filt=simplex_tree.get_filtration()
  *_, last0 = filt
  return last0[1]

In [16]:
def analizar_punto_gudhi(punto,puntos_ceros,puntos_unos,last0,last1):
  last_0=obtener_radio(np.array([punto.tolist()]+puntos_ceros.tolist()))
  last_1=obtener_radio(np.array([punto.tolist()]+puntos_unos.tolist()))


  if abs(last0-last_0)<0.00001 and abs(last1-last_1)>0.00001:
    clase=0
  elif abs(last1-last_1)<0.00001 and abs(last0-last_0)>0.00001:
    clase=1
  else:
    clase=-1
  return clase

In [17]:
def analizar_punto_gudhi2(punto,puntos_ceros,puntos_unos,last0,last1):
  
  last_0=obtener_radio(np.array([punto.tolist()]+puntos_ceros.tolist()))
  last_1=obtener_radio(np.array([punto.tolist()]+puntos_unos.tolist()))

  dif0=abs(last0-last_0)
  dif1=abs(last1-last_1)
  if dif0==0 and dif1==0:
    clase=-1
  elif dif0<dif1:
    clase=0
  else:
    clase=1
  return clase

In [18]:
def analizar_puntos_gudhi(data,target,func,reduccion=False):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target)
  embedding=data
  if reduccion==True:
    embedding=umap.UMAP(random_state=75).fit_transform(data)
  (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target)


  bien=0
  mal=0
  dudoso=0

  puntos_ceros_final=puntos_ceros.tolist()
  puntos_unos_final=puntos_unos.tolist()
  puntos_dudosos_final=[]

  last0=obtener_radio(puntos_ceros_umap)
  last1=obtener_radio(puntos_unos_umap)
  for i,punto in enumerate(X_unlabeled_unos_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,last0,last1)
    if clase==1:
      bien=bien+1
      puntos_unos_final.append(X_unlabeled_unos[i])
    elif clase==0:
      mal=mal+1
      puntos_ceros_final.append(X_unlabeled_unos[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_unos[i])


  
  for i,punto in enumerate(X_unlabeled_ceros_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,last0,last1)
    if clase==1:
      mal=mal+1
      puntos_unos_final.append(X_unlabeled_ceros[i])
    elif clase==0:
      bien=bien+1
      puntos_ceros_final.append(X_unlabeled_ceros[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_ceros[i])
  
  comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

  return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final))

In [19]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

def comprobar_accuracy(puntos_ceros_final,puntos_unos_final,X_unlabeled_ceros_valid,X_unlabeled_unos_valid):
    datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelSVMLineal = SVC(kernel="linear",probability=True,random_state=90)
    modelSVMLineal.fit(datos, labels)

    mal=np.count_nonzero(modelSVMLineal.predict(X_unlabeled_ceros_valid)!=0)
    mal=mal+np.count_nonzero(modelSVMLineal.predict(X_unlabeled_unos_valid)!=1)
    total=X_unlabeled_ceros_valid.shape[0]+X_unlabeled_unos_valid.shape[0]
    print("--------------Clasificador SVM--------------")
    resumen(total-mal,mal,0)
    print("")


    # datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    # labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelRandomForest = RandomForestClassifier(random_state=90)
    modelRandomForest.fit(datos, labels)

    mal=np.count_nonzero(modelRandomForest.predict(X_unlabeled_ceros_valid)!=0)
    mal=mal+np.count_nonzero(modelRandomForest.predict(X_unlabeled_unos_valid)!=1)
    total=X_unlabeled_ceros_valid.shape[0]+X_unlabeled_unos_valid.shape[0]
    print("-------------- Clasificador Random Forest--------------")
    resumen(total-mal,mal,0)
    print("")

In [20]:
def label_prop(data,target):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  label_prop_model = LabelPropagation(kernel='knn')
  label_prop_model.fit(datos, labels)

  unos=np.where(label_prop_model.predict(X_unlabeled_ceros)==1)[0]
  ceros=np.where(label_prop_model.predict(X_unlabeled_ceros)==0)[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  unos=np.where(label_prop_model.predict(X_unlabeled_unos)==1)[0]
  ceros=np.where(label_prop_model.predict(X_unlabeled_unos)==0)[0]
  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]

  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  mal=np.count_nonzero(label_prop_model.predict(X_unlabeled_ceros)!=0)
  mal=mal+np.count_nonzero(label_prop_model.predict(X_unlabeled_unos)!=1)
  total=X_unlabeled_ceros.shape[0]+X_unlabeled_unos.shape[0]
  # resumen(total-mal,mal)

  comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (total-mal, mal, 0, nuevos_puntos_cero, nuevos_puntos_uno, []) 

In [21]:
def label_spread(data,target):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  label_prop_model = LabelSpreading(kernel='knn')
  label_prop_model.fit(datos, labels)

  unos=np.where(label_prop_model.predict(X_unlabeled_ceros)==1)[0]
  ceros=np.where(label_prop_model.predict(X_unlabeled_ceros)==0)[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  unos=np.where(label_prop_model.predict(X_unlabeled_unos)==1)[0]
  ceros=np.where(label_prop_model.predict(X_unlabeled_unos)==0)[0]
  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]

  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  mal=np.count_nonzero(label_prop_model.predict(X_unlabeled_ceros)!=0)
  mal=mal+np.count_nonzero(label_prop_model.predict(X_unlabeled_unos)!=1)
  total=X_unlabeled_ceros.shape[0]+X_unlabeled_unos.shape[0]
  # resumen(total-mal,mal)

  comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (total-mal, mal, 0, nuevos_puntos_cero, nuevos_puntos_uno, []) 

In [22]:
def self_train(data,target):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  svc = SVC(probability=True, gamma="auto")
  self_training_model = SelfTrainingClassifier(svc)
  self_training_model.fit(datos, labels)

  unos=np.where(self_training_model.predict(X_unlabeled_ceros)==1)[0]
  ceros=np.where(self_training_model.predict(X_unlabeled_ceros)==0)[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  unos=np.where(self_training_model.predict(X_unlabeled_unos)==1)[0]
  ceros=np.where(self_training_model.predict(X_unlabeled_unos)==0)[0]
  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]

  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  mal=np.count_nonzero(self_training_model.predict(X_unlabeled_ceros)!=0)
  mal=mal+np.count_nonzero(self_training_model.predict(X_unlabeled_unos)!=1)
  total=X_unlabeled_ceros.shape[0]+X_unlabeled_unos.shape[0]
  # resumen(total-mal,mal)

  comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (total-mal, mal, 0, nuevos_puntos_cero, nuevos_puntos_uno, [])  

# Bottleneck

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 47
Mal anotados 13
Sin anotar 0

Porcentaje correcto 0.7833333333333333
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 193
Mal anotados 48
Sin anotar 0

Porcentaje correcto 0.8008298755186722
Porcentaje anotado 1.0


# Bottleneck threshold

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.8)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 47
Mal anotados 13
Sin anotar 0

Porcentaje correcto 0.7833333333333333
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 57
Mal anotados 3
Sin anotar 0

Porcentaje correcto 0.95
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 74
Mal anotados 0
Sin anotar 167

Porcentaje correcto 1.0
Porcentaje anotado 0.3070539419087137


# Bottleneck threshold 0.6

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.6)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 45
Mal anotados 15
Sin anotar 0

Porcentaje correcto 0.75
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 53
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.8833333333333333
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 181
Mal anotados 36
Sin anotar 24

Porcentaje correcto 0.8341013824884793
Porcentaje anotado 0.9004149377593361


# Bottleneck threshold 0.4

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.4)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 47
Mal anotados 13
Sin anotar 0

Porcentaje correcto 0.7833333333333333
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 193
Mal anotados 48
Sin anotar 0

Porcentaje correcto 0.8008298755186722
Porcentaje anotado 1.0


# Bottleneck threshold 0.2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.2)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 47
Mal anotados 13
Sin anotar 0

Porcentaje correcto 0.7833333333333333
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 193
Mal anotados 48
Sin anotar 0

Porcentaje correcto 0.8008298755186722
Porcentaje anotado 1.0


# Bottleneck UMAP

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto,0,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 51
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.85
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 53
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.8833333333333333
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 191
Mal anotados 50
Sin anotar 0

Porcentaje correcto 0.7925311203319502
Porcentaje anotado 1.0


# Bottleneck UMAP threshold

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.8,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 56
Mal anotados 4
Sin anotar 0

Porcentaje correcto 0.9333333333333333
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 97
Mal anotados 14
Sin anotar 130

Porcentaje correcto 0.8738738738738738
Porcentaje anotado 0.4605809128630705


# Bottleneck UMAP threshold 0.6

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.6,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 54
Mal anotados 6
Sin anotar 0

Porcentaje correcto 0.9
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 174
Mal anotados 41
Sin anotar 26

Porcentaje correcto 0.8093023255813954
Porcentaje anotado 0.8921161825726142


# Bottleneck UMAP threshold 0.4

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.4,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 51
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.85
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 53
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.8833333333333333
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 191
Mal anotados 50
Sin anotar 0

Porcentaje correcto 0.7925311203319502
Porcentaje anotado 1.0


# Bottleneck UMAP threshold 0.2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.2,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 51
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.85
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 53
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.8833333333333333
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 191
Mal anotados 50
Sin anotar 0

Porcentaje correcto 0.7925311203319502
Porcentaje anotado 1.0


# Wasserstein

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 46
Mal anotados 14
Sin anotar 0

Porcentaje correcto 0.7666666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 49
Mal anotados 11
Sin anotar 0

Porcentaje correcto 0.8166666666666667
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 189
Mal anotados 52
Sin anotar 0

Porcentaje correcto 0.7842323651452282
Porcentaje anotado 1.0


# Wasserstein threshold

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.8)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 48
Mal anotados 12
Sin anotar 0

Porcentaje correcto 0.8
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 57
Mal anotados 3
Sin anotar 0

Porcentaje correcto 0.95
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 60
Mal anotados 0
Sin anotar 181

Porcentaje correcto 1.0
Porcentaje anotado 0.24896265560165975


# Wasserstein threshold 0.6

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.6)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 42
Mal anotados 18
Sin anotar 0

Porcentaje correcto 0.7
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 55
Mal anotados 5
Sin anotar 0

Porcentaje correcto 0.9166666666666666
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 124
Mal anotados 15
Sin anotar 102

Porcentaje correcto 0.8920863309352518
Porcentaje anotado 0.5767634854771784


# Wasserstein threshold 0.4

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.4)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 46
Mal anotados 14
Sin anotar 0

Porcentaje correcto 0.7666666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 49
Mal anotados 11
Sin anotar 0

Porcentaje correcto 0.8166666666666667
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 189
Mal anotados 52
Sin anotar 0

Porcentaje correcto 0.7842323651452282
Porcentaje anotado 1.0


# Wasserstein threshold 0.2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.2)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 46
Mal anotados 14
Sin anotar 0

Porcentaje correcto 0.7666666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 49
Mal anotados 11
Sin anotar 0

Porcentaje correcto 0.8166666666666667
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 189
Mal anotados 52
Sin anotar 0

Porcentaje correcto 0.7842323651452282
Porcentaje anotado 1.0


# Wasserstein UMAP

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein,0,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 47
Mal anotados 13
Sin anotar 0

Porcentaje correcto 0.7833333333333333
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 55
Mal anotados 5
Sin anotar 0

Porcentaje correcto 0.9166666666666666
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 186
Mal anotados 55
Sin anotar 0

Porcentaje correcto 0.7717842323651453
Porcentaje anotado 1.0


# Wasserstein UMAP threshold

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.8,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 51
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.85
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 56
Mal anotados 4
Sin anotar 0

Porcentaje correcto 0.9333333333333333
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 75
Mal anotados 11
Sin anotar 155

Porcentaje correcto 0.872093023255814
Porcentaje anotado 0.35684647302904565


# Wasserstein UMAP threshold 0.6

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.6,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 54
Mal anotados 6
Sin anotar 0

Porcentaje correcto 0.9
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 165
Mal anotados 42
Sin anotar 34

Porcentaje correcto 0.7971014492753623
Porcentaje anotado 0.8589211618257261


# Wasserstein UMAP threshold 0.4

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.4,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 47
Mal anotados 13
Sin anotar 0

Porcentaje correcto 0.7833333333333333
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 55
Mal anotados 5
Sin anotar 0

Porcentaje correcto 0.9166666666666666
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 186
Mal anotados 55
Sin anotar 0

Porcentaje correcto 0.7717842323651453
Porcentaje anotado 1.0


# Wasserstein UMAP threshold 0.2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.2,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 47
Mal anotados 13
Sin anotar 0

Porcentaje correcto 0.7833333333333333
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 55
Mal anotados 5
Sin anotar 0

Porcentaje correcto 0.9166666666666666
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 186
Mal anotados 55
Sin anotar 0

Porcentaje correcto 0.7717842323651453
Porcentaje anotado 1.0


# Gudhi

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 46
Mal anotados 14
Sin anotar 0

Porcentaje correcto 0.7666666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 53
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.8833333333333333
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 2
Mal anotados 11
Sin anotar 228

Porcentaje correcto 0.15384615384615385
Porcentaje anotado 0.05394190871369295


# Gudhi UMAP

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 50
Mal anotados 10
Sin anotar 0

Porcentaje correcto 0.8333333333333334
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 60
Mal anotados 0
Sin anotar 0

Porcentaje correcto 1.0
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 35
Mal anotados 3
Sin anotar 203

Porcentaje correcto 0.9210526315789473
Porcentaje anotado 0.15767634854771784


# Gudhi 2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi2)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 43
Mal anotados 17
Sin anotar 0

Porcentaje correcto 0.7166666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 50
Mal anotados 10
Sin anotar 0

Porcentaje correcto 0.8333333333333334
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 4
Mal anotados 19
Sin anotar 218

Porcentaje correcto 0.17391304347826086
Porcentaje anotado 0.0954356846473029


# Gudhi 2 UMAP

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi2,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 48
Mal anotados 12
Sin anotar 0

Porcentaje correcto 0.8
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 57
Mal anotados 3
Sin anotar 0

Porcentaje correcto 0.95
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 50
Mal anotados 4
Sin anotar 187

Porcentaje correcto 0.9259259259259259
Porcentaje anotado 0.22406639004149378


# LabelPropagation

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=label_prop(data,target)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 197
Mal anotados 44
Sin anotar 0

Porcentaje correcto 0.8174273858921162
Porcentaje anotado 1.0


# LabelSpreading

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=label_spread(data,target)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 197
Mal anotados 44
Sin anotar 0

Porcentaje correcto 0.8174273858921162
Porcentaje anotado 1.0


# SelfTraining

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=self_train(data,target)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 51
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.85
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 52
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.8666666666666667
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 200
Mal anotados 41
Sin anotar 0

Porcentaje correcto 0.8298755186721992
Porcentaje anotado 1.0


# Base

In [23]:
(data,target) = load_prima()
(puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target)
comprobar_accuracy(puntos_ceros,puntos_unos,puntos_ceros_valid,puntos_unos_valid)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 48
Mal anotados 12
Sin anotar 0

Porcentaje correcto 0.8
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 56
Mal anotados 4
Sin anotar 0

Porcentaje correcto 0.9333333333333333
Porcentaje anotado 1.0

